<b>Blending: Gasoline blending</b>

In [2]:
# Data on products:
Products = {'Regular', 'Plus', 'Premium'}
MinRating = {'Regular' : 87, 'Plus' : 89, 'Premium' : 93 }
MinDeliver = {'Regular' : 60000, 'Plus' : 25000, 'Premium' : 15000}
MaxDeliver = {'Regular' : 80000, 'Plus' : 70000, 'Premium' : 40000}
Price = {'Regular' : 48, 'Plus' : 50, 'Premium' : 53}

# Data on components:
Components = {'C1', 'C2', 'C3', 'C4'}
Rating = {'C1' : 86, 'C2' : 96, 'C3' : 85, 'C4' : 94}
Supply = {'C1' : 70000, 'C2' : 50000, 'C3' : 40000, 'C4' : 30000}
Cost = {'C1' : 33, 'C2' : 37, 'C3' :  32, 'C4' : 35}

In [1]:
from docplex.mp.model import Model
mdl = Model()

In [3]:
# variables
InBlend = mdl.continuous_var_matrix(Components, Products, lb=0, name='InBlend')

In [4]:
#Objective

TotalRevenue = mdl.sum(InBlend[(c,p)] * Price[p] for c in Components for p in Products) 
TotalCost = mdl.sum(InBlend[(c,p)] * Cost[c] for c in Components for p in Products) 
TotalProfit = TotalRevenue - TotalCost
mdl.maximize(TotalProfit)

In [5]:
mdl.add_kpi(TotalRevenue, 'Total Revenue')
mdl.add_kpi(TotalCost, 'Total Cost')
mdl.add_kpi(TotalProfit, 'Total Profit')

DecisionKPI(name=Total Profit,expr=18InBlend_C4_Premium+13InBlend_C4_Regular+15InBlend_C4_Plus+21In..)

In [7]:
#min & max
for p in Products:
    mdl.add_range(MinDeliver[p], mdl.sum(InBlend[(c,p)] for c in Components), MaxDeliver[p])

In [8]:
#Supply
for c in Components:
    mdl.add_constraint(mdl.sum(InBlend[(c,p)] for p in Products) <= Supply[c])

In [9]:
# min octane rating (blending constraints)
for p in Products:
    mdl.add_constraint(mdl.sum((Rating[c]-MinRating[p])*InBlend[(c,p)] for c in Components) >= 0)

In [10]:
mdl.solve()

docplex.mp.solution.SolveSolution(obj=2.97e+06,values={InBlend_C4_Plus:3..

In [11]:
mdl.get_solve_details()

docplex.mp.SolveDetails(time=0,status='optimal')

In [12]:
mdl.print_solution()

objective: 2970000.000
status: OPTIMAL_SOLUTION(2)
  InBlend_C4_Plus=30000.000
  InBlend_C3_Regular=909.091
  InBlend_C3_Plus=39090.909
  InBlend_C2_Premium=28000.000
  InBlend_C2_Regular=21090.909
  InBlend_C2_Plus=909.091
  InBlend_C1_Premium=12000.000
  InBlend_C1_Regular=58000.000


In [13]:
mdl.report()

* model docplex_model1 solved with objective = 2970000.000
*  KPI: Total Revenue = 9460000.000
*  KPI: Total Cost    = 6490000.000
*  KPI: Total Profit  = 2970000.000


In [14]:
# add octane ratings as KPI
OctaneRating = {}
for p in Products:
    OctaneRating[p] = sum(Rating[c]*InBlend[(c,p)].solution_value for c in Components)/sum(InBlend[(c,p)].solution_value for c in Components)
    mdl.add_kpi(OctaneRating[p], p)

In [15]:
mdl.report()

* model docplex_model1 solved with objective = 2970000.000
*  KPI: Total Revenue = 9460000.000
*  KPI: Total Cost    = 6490000.000
*  KPI: Total Profit  = 2970000.000
*  KPI: Premium       = 93.000
*  KPI: Regular       = 88.625
*  KPI: Plus          = 89.000
